#1. Projeto 4 do curso de pytorch

##2. Importar as bibliotecas

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MaxAbsScaler
import torch as tt
import torch.nn as nn

##3. Deixar o fator randomico de reprodutibilidade em 1

In [2]:
np.random.seed(1)
tt.manual_seed(1)

##4. Importar as bases de dados

In [13]:
previsores = pd.read_csv('/content/drive/MyDrive/entradas_breast.csv')
predito = pd.read_csv('/content/drive/MyDrive/saidas_breast.csv')
print(predito.shape)
quantidade_unico = np.unique(predito)
print('Classes possíveis: {}'.format(quantidade_unico))

(569, 1)
Classes possíveis: [0 1]


##5. Redefinindo as variaveis para numpy

In [14]:
previsoes = np.array(previsores,dtype='float32')
classes = np.array(predito,dtype='float32')#tira a dimensão extra do array

##6. Normalizando as entradas

In [15]:
previsoes_norm = MaxAbsScaler().fit(previsoes)
previsoes_norm = previsoes_norm.transform(previsoes)

##7. Redefinindo as variaveis para torch

In [16]:
previsoes_norm = tt.tensor(previsoes_norm,dtype=tt.float)
classes = tt.tensor(classes,dtype=tt.float)

##8. Construir os dados e a rede com os parametros testados na fase 3
1. 'batch_size': 20, 
2. 'criterion': <class 'torch.nn.modules.loss.BCELoss'>,
3.'lr': 0.005, 
4.'max_epochs': 100, 
5.'module__activation': <built-in method tanh of type object at 0x7f19b64a61a0>, 
6.'module__initializer': <function zeros_ at 0x7f18b9559b00>, 
7.'module__neurons': 16, 
8.'optimizer': <class 'torch.optim.adam.Adam'>

##9. Criar um dataset e um dataloader 

In [17]:
#construir o dataset pytorch
dataset = tt.utils.data.TensorDataset(previsoes_norm,classes)
#construir o microbatch de treinamento
dataloader = tt.utils.data.DataLoader(dataset,batch_size=20,shuffle=True)

##10. Construir a rede

In [53]:
class classificador(nn.Module):
  def __init__(self):
    super().__init__()
    self.camada0 = nn.Linear(in_features = 30, out_features = 16)
    tt.nn.init.zeros_(self.camada0.weight)
    self.ativacao0 = nn.Tanh()
    self.dp0 = nn.Dropout(0.2)
    self.camada1 = nn.Linear(in_features = 16, out_features = 16)
    tt.nn.init.zeros_(self.camada1.weight)
    self.ativacao1 = nn.Tanh()
    self.dp1 = nn.Dropout(0.2)
    self.camada2 = nn.Linear(in_features=16, out_features=1)
    tt.nn.init.zeros_(self.camada2.weight)
    self.output = nn.Sigmoid()
  def forward(self, x):
    x = self.camada0(x)
    x = self.ativacao0(x)
    x = self.dp0(x)
    x = self.camada1(x)
    x = self.ativacao1(x)
    x = self.dp1(x)
    x = self.camada2(x)
    x = self.output(x)
    return x

##11. Aplicar os hiperparametros e função de perda

In [19]:
classificador = classificador()
#função de erro 
er = nn.BCELoss()
#otimizadores
otm = tt.optim.Adam(classificador.parameters(), lr = 0.005, weight_decay=0.0001)

##12. Fazer o treinamento

In [ ]:
#treinamento
for epoch in range(100):
    ls = 0.0
    for data in dataloader:
        inputs, labels = data #separa as entradas e as saidas desejadas
        otm.zero_grad() #zera todos os gradientes
        outputs = classificador.forward(inputs) #faz o passo pra frente
        loss = er(outputs,labels) #Calcula o erro
        loss.backward() #reajusta os pesos
        otm.step() #calcula o gradiente
        ls = ls+loss.item()# Calcula o erro global
    print('Epoca: {}, Erro: {}'.format(epoch,ls/len(dataloader)))

##13. Classificar um novo atributo unico

In [26]:
novo = tt.tensor([[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]],dtype=tt.float)

##14. Classificador em modo de avaliação

In [22]:
classificador.eval()

classificador(
  (camada0): Linear(in_features=30, out_features=16, bias=True)
  (ativacao0): Tanh()
  (dp0): Dropout(p=0.2, inplace=False)
  (camada1): Linear(in_features=16, out_features=16, bias=True)
  (ativacao1): Tanh()
  (dp1): Dropout(p=0.2, inplace=False)
  (camada2): Linear(in_features=16, out_features=1, bias=True)
  (output): Sigmoid()
)

##15. Fazer o foward

In [43]:
previsao = classificador.forward(novo)

##16. Ve se o tumor é maligno

In [45]:
previsao = previsao.detach()
previsao = previsao.numpy()
previsao = (previsao > 0.5)
print('O tumor é maligno: {}'.format(previsao))

O tumor é maligno: [[False]]


##17. Salvar o classificador

In [51]:
tt.save(classificador.state_dict(),'classificador.pth')

##18. Carregar os pesos

In [54]:
cla = classificador() #É necessário refazer toda a arquitetura da rede e depois instanciar de novo, nesse caso
                      # eu só reaproveitei o que ja tinha criado nesse arquivo
std = tt.load('/content/classificador.pth')#Aqui faz o load dos pesos
cla.load_state_dict(std)#Agora atualiza os pesos

##Fazer uma previsão

In [58]:
novo = tt.tensor([[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]],dtype=tt.float)
cla.eval()
previsao = cla.forward(novo)
previsao = previsao.detach()
previsao = previsao.numpy()
previsao = (previsao > 0.5)
print('O tumor é maligno: {}'.format(previsao))

O tumor é maligno: [[False]]
